## Simple linear regession regularization

In [ ]:
from abc import ABCMeta
import six
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.base import RegressorMixin
from sklearn.linear_model.base import LinearModel
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

# set random seed to have consistent result
np.random.seed(1)

%matplotlib inline

### Helper function

In [ ]:
def compute_rmse(target, pred):
    """Compute RMSE between predict and true target
    """
    return np.sqrt(mean_squared_error(target, pred))


def create_poly_feature_pipe(degree, normalize=True):
    """Creaet poly feature transform pipeline
    """
    poly = PolynomialFeatures(degree, include_bias=False)
    
    if normalize:
        scaler = StandardScaler()
        pipe = make_pipeline(poly, scaler)
    else:
        pipe = make_pipeline(poly)
        
    return pipe
        

def extract_model_weights(model):
    """Extract weights from model
    """
    if hasattr(model, 'intercept_'):
        w = np.zeros(len(model.coef_.ravel()) + 1)
        w[0] = model.intercept_
        w[1:] = model.coef_.ravel()
    else:
        w = model.coef_.copy()
        
    return w
    
    
def print_model_weight(model):    
    print('weights:\n', np.array2string(extract_model_weights(model).ravel(), 
                        formatter={'float_kind':'{:2.6e}'.format}, max_line_width=80))

def print_model_rmse(model, preprocess_pipe, data_set, string_format='RMSE: {:2.6e}'):
    X, y  = data_set
    X_tr = preprocess_pipe.transform(X)
    rmse = compute_rmse(y, model.predict(X_tr))
    print(string_format.format(rmse))


def show_model_result(model, preprocess_pipe, train_set, valid_set=None, test_set=None):
    """Print linear regression model RMSE and weights
    
    :param model: linear regression model
    :param preprocess_pipe:  preprocess pipeline 
    :param train_set: (X, y) tuple train data X, y 
    :param valid_set: (X, y) tuple valid data X, y
    :param test_set:  (X, y) tuple test  data X, y
    """
    
    print_model_rmse(model, preprocess_pipe, train_set, 'Train RMSE: {:2.6e}')
    
    if valid_set is not None:
        print_model_rmse(model, preprocess_pipe, valid_set, 'Valid RMSE: {:2.6e}')
    
    if test_set is not None:
        print_model_rmse(model, preprocess_pipe, test_set, 'Test RMSE: {:2.6e}')
    
    print_model_weight(model)
    print('='*80)


def plot_model_result(model, preprocess_pipe, train_set, valid_set=None, test_set=None):
    """Plot linear regression model result
    """
    
    def plot_data(data_set, color, label):
        X, y = data_set
        plt.scatter(X, y, color=color, s=20, alpha=0.5, label=label)
    
    plt.figure(figsize=(5, 5))
        
    plot_data(train_set, color='b', label='Train')

    if valid_set is not None:
        plot_data(valid_set, color='g', label='Valid')
        
    if test_set is not None:
        plot_data(test_set, color='y', label='Test')
    
    train_X, train_y = train_set
    plot_X = np.linspace(train_X.min(), train_X.max(), 1000).reshape([-1, 1])
    plot_X_tr = preprocess_pipe.transform(plot_X)
    plot_y_pred = model.predict(plot_X_tr)
    plt.plot(plot_X, plot_y_pred, label='model', linewidth=3, color='r')
    
    plt.title('lambda: {:.2e}'.format(model.alpha))
    plt.xlim(xmin=train_X.min(), xmax=train_X.max())
    plt.ylim(ymin=train_y.min(), ymax=train_y.max())
    plt.xlabel('X', fontsize='large')
    plt.ylabel('y', rotation='horizontal', fontsize='large')
    plt.legend(loc='upper right')
    plt.show()

### Linear Regression cost and gradient function

In [ ]:
def compute_cost(X, y, w, lambda_):
    """Linear regression cost function with l2-norm regularization
    """
    n_samples, n_features = X.shape
    
    y_hat = X @ w
    diff = y_hat - y
    
    ww = w.copy()
    ww[0] = 0
    cost = (diff.T @ diff + lambda_ * ww.T @ ww) / (2 * n_samples) 
    
    return cost

    
def compute_grad(X, y, w, lambda_):
    """Linear regression grad function with l2-nrom regularization
    """
    n_samples, n_features = X.shape
    y_hat = X @ w
    diff = y_hat - y

    # compute grad
    cost_grad = X.T @ diff
        
    # compute regularization grad
    mask = np.ones([n_features, 1])
    mask[0] = 0
    reg_grad = lambda_ * (w * mask) 
    grad = (cost_grad + reg_grad) / n_samples
          
    return grad

### Base Solver

In [ ]:
class _BaseSolver(six.with_metaclass(ABCMeta, LinearModel)):
    def predict(self, X, y=None):
        X, y = preprocess_data(X, y, self.add_bias)
        return X @ self.coef_

#    def rmse(self, X, y):
#        return rmse(self.predict(X) - y)
#    
#    def cost(self, X, y):
#        return compute_cost(y, self.predict(X))


def preprocess_data(X, y, add_bias):
    """Preprocess X, y data with add bias term to X
    
    :param X: {n_samples, n_featurs}
    :param y: {n_samples}
    :param add_bias: if True add bias term to X
    :return: X, y
    """
    X = X.copy()
    
    if add_bias:
        bias = np.ones([X.shape[0], 1])
        X = np.hstack([bias, X])
    
    return X, y

### Gradient Descent Solver

In [ ]:
def gradient_descent_solver(X, y, lambda_=1, learning_rate=1e-2, tol=1e-8, max_iter=None):
    """Solve Xw = y using gradient desecent method with l2-norm regularization
    
    :param X: {n_samples, n_featurs}
    :param y: {n_samples} 
    :param lambda_: l2-norm regularization penalty
    :param learning_rate: gradent step 
    :param tol: tolerance between cost funciton
    :param max_iter: max iterations number
    :return: {n_samples} coffecient
    """
    n_samples, n_features = X.shape
    w = np.zeros([n_features, 1])
    
    mask = np.ones([n_features, 1])
    mask[0] = 0
    
    if max_iter is None:
        max_iter = 1000
    costs = np.zeros(max_iter)
    
    costs[0] = compute_cost(X, y, w, lambda_)
    for n_iter in np.arange(1, max_iter):    
        y_hat = X @ w
        diff = y_hat - y
    
        grad = compute_grad(X, y, w, lambda_)
        w = w - learning_rate * grad
        
        costs[n_iter] = compute_cost(X, y, w, lambda_)
        
        #print('costs[%4d]: %.8f' % (n_iter, costs[n_iter]))
        if np.abs(costs[n_iter - 1] - costs[n_iter]) < tol:
            costs = costs[:n_iter]
            break
    
    return w, n_iter, costs
    

# from scipy.optimize import fmin_cg
# 
# class CGSolver(_BaseSolver, RegressorMixin):
#     """USing scipy.optimize.fmin_cg to optimize linear regression model
#     """
#     def __init__(self, alpha=1.0, add_bias=True, max_iter=None, tol=1e-6):
#         self.alpha = alpha
#         self.add_bias = add_bias
#         self.max_iter = max_iter
#         self.tol = tol
#         
#     def fit(self, X, y):
#         X, y = preprocess_data(X, y, self.add_bias)
# 
#         def cost_fun(w, X, y, lambda_):
#             w = np.reshape(w, [-1, 1])
#             return compute_cost(X, y, w, lambda_)
#         
#         def grad_fun(w, X, y, lambda_):
#             w = np.reshape(w, [-1, 1])
#             grad = compute_grad(X, y, w, lambda_)
#             
#             return grad.flatten()
#         
#         x0 = np.zeros([X.shape[1]])
#         self.coef_ = \
#             fmin_cg(cost_fun, x0, fprime=grad_fun, args=(X, y, self.alpha), 
#                     maxiter=self.max_iter, epsilon=self.tol,
#                     disp=True)
#         self.coef_ = self.coef_.T
        
    
class GradientDescentSolver(_BaseSolver, RegressorMixin):
    """Gradient descent solver with l2 regularization
    
    This model solves a regression model with the loss function using
    gradient descent method with l2-nrom regularization
    """
    
    def __init__(self, alpha=1.0, add_bias=True, learning_rate=1e-2, max_iter=None, tol=1e-8):
        self.alpha = alpha
        self.add_bias = add_bias
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.tol = tol
        
    def fit(self, X, y):
        X, y = preprocess_data(X, y, self.add_bias)
        
        self.coef_, self.n_iters_, self.costs_ = \
            gradient_descent_solver(X, y, 
                                    lambda_=self.alpha, learning_rate=self.learning_rate,
                                    max_iter=self.max_iter, tol=self.tol)

## 1.Polynomial regression / overfitting / regularization

### Fit the data using linear (1st order) regression model (gradient descent method)

In [ ]:
# Prepare data
df = pd.read_csv('ex2data1.csv')
data1_X = df.iloc[:, :-1].values.reshape([-1, 1])
data1_y = df.iloc[:, -1].values.reshape([-1, 1])

data1_train_set = (data1_X, data1_y)

In [ ]:
# parameters
degree = 1
lambda_ = 0

# create preprocess pipe line
preprocess_pipe = create_poly_feature_pipe(degree, normalize=True)
data1_X_tr = preprocess_pipe.fit_transform(data1_X)
    
model = GradientDescentSolver(lambda_)
model.fit(data1_X_tr, data1_y)

plot_model_result(model, preprocess_pipe, data1_train_set)
show_model_result(model, preprocess_pipe, data1_train_set)

### Fit the data using 2nd order polynomial regression model (gradient descent method).

In [ ]:
# parameters
degree = 2
lambda_ = 0

# create preprocess pipe line
preprocess_pipe = create_poly_feature_pipe(degree, normalize=True)
data1_X_tr = preprocess_pipe.fit_transform(data1_X)
    
model = GradientDescentSolver(lambda_)
model.fit(data1_X_tr, data1_y)

plot_model_result(model, preprocess_pipe, data1_train_set)
show_model_result(model, preprocess_pipe, data1_train_set)

### Fit the data using 4th order polynomial regression model (gradient descent method).

In [ ]:
# parameters
degree = 4
lambda_ = 0

# create preprocess pipe line
preprocess_pipe = create_poly_feature_pipe(degree, normalize=True)
data1_X_tr = preprocess_pipe.fit_transform(data1_X)
    
model = GradientDescentSolver(lambda_, learning_rate=0.1, max_iter=8000)
model.fit(data1_X_tr, data1_y)

plot_model_result(model, preprocess_pipe, data1_train_set)
show_model_result(model, preprocess_pipe, data1_train_set)

### Fit the data using 16th order polynomial regression model (gradient descent method).

With 16 degree poly features, but only have very small sample data (aka a under-fitting system)

This caused matrix ***X*** very ill-conditioned.

Without L2-norm regularization (lambda_ = 0), matrix form result is very unstable

In [ ]:
# parameters
degree = 16
lambda_ = 0

# create preprocess pipe line
preprocess_pipe = create_poly_feature_pipe(degree, normalize=True)
data1_X_tr = preprocess_pipe.fit_transform(data1_X)
    
model = GradientDescentSolver(lambda_, learning_rate=0.1, max_iter=2000)
model.fit(data1_X_tr, data1_y)

plot_model_result(model, preprocess_pipe, data1_train_set)
show_model_result(model, preprocess_pipe, data1_train_set)

### Fit the data using 16th order polynomial regression model with ridge (L2 penalty) regularization (gradient descent method).

In [ ]:
# parameters
degree = 16

# create preprocess pipe line
preprocess_pipe = create_poly_feature_pipe(degree, normalize=True)
data1_X_tr = preprocess_pipe.fit_transform(data1_X)
    
lambda_vec = [np.power(10.0, p) for p in np.arange(-1, 2, 1)]

for lambda_ in lambda_vec:

    model = GradientDescentSolver(lambda_, learning_rate=0.1, max_iter=3000)
    model.fit(data1_X_tr, data1_y)

    plot_model_result(model, preprocess_pipe, data1_train_set)
    show_model_result(model, preprocess_pipe, data1_train_set)

### Fit the data using 16th order polynomial regression model with scikit-learn Ridge model.

In [ ]:
from sklearn.linear_model import Ridge

# parameters
degree = 16

# create preprocess pipe line
preprocess_pipe = create_poly_feature_pipe(degree, normalize=True)
data1_X_tr = preprocess_pipe.fit_transform(data1_X)

lambda_vec = [np.power(10.0, p) for p in np.arange(-1, 2, 1)]

for lambda_ in lambda_vec:

    model = Ridge(lambda_)
    model.fit(data1_X_tr, data1_y)

    plot_model_result(model, preprocess_pipe, data1_train_set)
    show_model_result(model, preprocess_pipe, data1_train_set)

### Fit the data using 16th order polynomial regression model with scikit-learn Lasso model.

L1-norm regularization tend to sparse solution, so the most weights compressed

In [ ]:
from sklearn.linear_model import Lasso

# create preprocess pipe line
preprocess_pipe = create_poly_feature_pipe(degree, normalize=True)
data1_X_tr = preprocess_pipe.fit_transform(data1_X)

# parameters
degree = 16

lambda_vec = [np.power(10.0, p) for p in np.arange(-1, 2, 1)]

for lambda_ in lambda_vec:

    model = Lasso(lambda_)
    model.fit(data1_X_tr, data1_y)

    plot_model_result(model, preprocess_pipe, data1_train_set)
    show_model_result(model, preprocess_pipe, data1_train_set)

## 2.Polynomial regression with train/validation/test

In [ ]:
def train_val_test_split(data, train_ratio, valid_ratio, test_ratio):
    """Split with ratio
    """
    shuffle_idx = np.random.permutation(len(data))
    
    train_size = int(len(data) * train_ratio)
    valid_size = int(len(data) * valid_ratio)
    test_size  = int(len(data) * test_ratio)
    
    train_idx = shuffle_idx[:train_size]
    valid_idx = shuffle_idx[(train_size):(train_size + valid_size)]
    test_idx  = shuffle_idx[(train_size + valid_size):]

    return data.iloc[train_idx], data.iloc[valid_idx], data.iloc[test_idx]

In [ ]:
df = pd.read_csv('ex2data2.csv')
split_train, split_valid, split_test = train_val_test_split(df, 0.6, 0.2, 0.2)

data2_train_X, data2_train_y = split_train.iloc[:, 0].values.reshape(-1, 1), \
                               split_train.iloc[:, -1].values.reshape(-1, 1)
data2_valid_X, data2_valid_y = split_valid.iloc[:, 0].values.reshape(-1, 1), \
                               split_valid.iloc[:, -1].values.reshape(-1, 1)
data2_test_X ,  data2_test_y = split_test.iloc[:, 0].values.reshape(-1, 1), \
                               split_test.iloc[:, -1].values.reshape(-1, 1)

data2_train_set = (data2_train_X, data2_train_y)
data2_valid_set = (data2_valid_X, data2_valid_y)
data2_test_set  = (data2_test_X ,  data2_test_y)

### Fit the data using linear (1st order) regression model

In [ ]:
# parameters
degree = 1
lambda_ = 0

# create preprocess pipe line
preprocess_pipe = create_poly_feature_pipe(degree, normalize=True)
data2_train_X_tr = preprocess_pipe.fit_transform(data2_train_X)
    
model = GradientDescentSolver(lambda_)
model.fit(data2_train_X_tr, data2_train_y)

plot_model_result(model, preprocess_pipe, data2_train_set)
show_model_result(model, preprocess_pipe, data2_train_set, test_set=data2_test_set)

### Fit the data using 2nd order polynomial regression model

In [ ]:
# parameters
degree = 2
lambda_ = 0

# create preprocess pipe line
preprocess_pipe = create_poly_feature_pipe(degree, normalize=True)
data2_train_X_tr = preprocess_pipe.fit_transform(data2_train_X)
    
model = GradientDescentSolver(lambda_)
model.fit(data2_train_X_tr, data2_train_y)

plot_model_result(model, preprocess_pipe, data2_train_set)
show_model_result(model, preprocess_pipe, data2_train_set, test_set=data2_train_set)

### Fit the data using 4th order polynomial regression model

In [ ]:
# parameters
degree = 4
lambda_ = 0

# create preprocess pipe line
preprocess_pipe = create_poly_feature_pipe(degree, normalize=True)
data2_train_X_tr = preprocess_pipe.fit_transform(data2_train_X)
    
model = GradientDescentSolver(lambda_)
model.fit(data2_train_X_tr, data2_train_y)

plot_model_result(model, preprocess_pipe, data2_train_set)
show_model_result(model, preprocess_pipe, data2_train_set, test_set=data2_test_set)

### Fit the data using 16th order polynomial regression model

In [ ]:
# parameters
degree = 16
lambda_ = 0

# create preprocess pipe line
preprocess_pipe = create_poly_feature_pipe(degree, normalize=True)
data2_train_X_tr = preprocess_pipe.fit_transform(data2_train_X)
    
model = GradientDescentSolver(lambda_)
model.fit(data2_train_X_tr, data2_train_y)

plot_model_result(model, preprocess_pipe, data2_train_set)
show_model_result(model, preprocess_pipe, data2_train_set, test_set=data2_test_set)

### Fit the data using 16th order polynomial regression model with ridge

#### To select optimal $\lambda$ by compute different rmse for test and valid

In [ ]:
def compute_validate_curve(model, preprocess_pipe, 
                           train_set, valid_set, lambda_vec):
    """Compute different lambda_ value affect RMSE for train and valid set
    """
    train_rmse = np.zeros(len(lambda_vec))
    valid_rmse = np.zeros(len(lambda_vec))
    weights = []
    
    train_X, train_y = train_set
    valid_X, valid_y = valid_set
    
    train_X_tr = preprocess_pipe.transform(train_X)
    valid_X_tr = preprocess_pipe.transform(valid_X)
    
    for idx, lambda_ in enumerate(lambda_vec):
        
        # dirty hack to chage model regularization penalty
        model.alpha = lambda_
        model.fit(train_X_tr, train_y)
        weights.append(extract_model_weights(model))
        
        train_rmse[idx] = compute_rmse(train_y, model.predict(train_X_tr))
        valid_rmse[idx] = compute_rmse(valid_y, model.predict(valid_X_tr))
    
    weights = np.squeeze(np.array(weights))
    
    return train_rmse, valid_rmse, weights

def plot_validate_curve(train_rmse, valid_rmse, lambda_vec):
    """Plot different lambda on train and valid result
    """
    plt.figure(figsize=(8, 5))
    plt.plot(lambda_vec, train_rmse, color='g', alpha=0.5, label='Train')
    plt.plot(lambda_vec, valid_rmse, color='b', alpha=0.5, label='Valid')
    plt.title('$\lambda$ curve')
    plt.xlim(xmin=lambda_vec.min(), xmax=lambda_vec.max())
    plt.ylim(ymax=max(train_rmse.max(), valid_rmse.max()))
    plt.xlabel('$\lambda$', fontsize='large')
    plt.ylabel('RMSE', fontsize='large')
    plt.legend(loc='upper right')
    plt.show()
    
    
def plot_lambda_weights_effect(weights, lambda_vec, weights_name):
    """Plot the weights value change with different lambda
    
    :param weights: [n_lambda, n_feature] 
    :param lambda_vec: [n_lambda] 
    :param weights_name: list of weight name with length n_lambda
    """
    
    plt.figure(figsize=(8, 5))
    n_lambda, n_features = weights.shape
    
    for n in range(n_features):
        w = weights[:, n]
        label_name = weights_name[n]
        plt.plot(lambda_vec, w, label=label_name, alpha=0.7)

    plt.title('weights regularization by lambda')
    plt.xlabel('$\lambda$')
    plt.ylabel('weight')

    plt.legend(loc="upper right")

#### Gradient Descent

In [ ]:
# parameters
degree = 16

grid = np.linspace(-2, 5, 50)
lambda_vec = np.array([0] + [np.power(10.0, p) for p in grid])
#lambda_vec = np.r_[0, np.logspace(-2, 5, 20)].reshape(-1, 1)

train_rmse = np.zeros(len(lambda_vec))
valid_rmse = np.zeros(len(lambda_vec))

# FIXME: GradientDescentSolver get wrong result in compute_validate_curve
#preprocess_pipe = create_poly_feature_pipe(degree, normalize=True)
#preprocess_pipe.fit(X_train)
#model = GradientDescentSolver(learning_rate=0.01, max_iter=8000)
#train_rmse, valid_rmse = compute_validate_curve(model, preprocess_pipe, train_set, valid_set, lambdas)
#plot_validate_curve(train_rmse, valid_rmse, lambdas)

# create preprocess pipe line
preprocess_pipe = create_poly_feature_pipe(degree, normalize=True)
data2_train_X_tr = preprocess_pipe.fit_transform(data2_train_X)
data2_valid_X_tr = preprocess_pipe.transform(data2_valid_X)

weights = []
for idx, lambda_ in enumerate(lambda_vec):

    model = GradientDescentSolver(lambda_, learning_rate=0.01, max_iter=8000)
    model.fit(data2_train_X_tr, data2_train_y)
    weights.append(extract_model_weights(model))
    
    train_rmse[idx] = compute_rmse(data2_train_y, model.predict(data2_train_X_tr))
    valid_rmse[idx] = compute_rmse(data2_valid_y, model.predict(data2_valid_X_tr))

weights = np.squeeze(np.array(weights))

plot_validate_curve(train_rmse, valid_rmse, lambda_vec)

#### Gradient Descent with ridge (L2 penalty) regularization $\lambda$ on weights effect

In [ ]:
# plot weights with lambda regularization
weights_name = ['{:2d} th poly'.format(d) for d in np.arange(1, degree + 1)]
weights = weights[:, 1:] # remove bias term
plot_lambda_weights_effect(weights, lambda_vec, weights_name)

#### Gradient Decent best $\lambda$ result

In [ ]:
# find lambda with lowest train and valid rmse, skip lambda = 0
opt_lambda = lambda_vec[np.argmin(np.abs(train_rmse[1:] - valid_rmse[1:])) + 1]
data2_train_X_tr = preprocess_pipe.transform(data2_train_X)
model.alpha = opt_lambda
model.fit(data2_train_X_tr, data2_train_y)

print("optimal lambda {:2.6e}".format(opt_lambda))
plot_model_result(model, preprocess_pipe, data2_train_set, data2_valid_set)
show_model_result(model, preprocess_pipe, data2_train_set, data2_valid_set, data2_test_set)

### Fit the data using 16th order polynomial regression model with scikit-learn Ridge model.

In [ ]:
# parameters
degree = 16

grid = np.linspace(-2, 5, 50)
lambda_vec = np.array([0] + [np.power(10.0, p) for p in grid])
#lambda_vec = np.r_[0, np.logspace(-2, 5, 20)].reshape(-1, 1)

preprocess_pipe = create_poly_feature_pipe(degree, normalize=True)
preprocess_pipe.fit(data2_train_X)

model = Ridge(solver='lsqr')
train_rmse, valid_rmse, weights = compute_validate_curve(model, preprocess_pipe, 
                                                data2_train_set, data2_valid_set, lambda_vec)
plot_validate_curve(train_rmse, valid_rmse, lambda_vec)

#### scikit-learn Ridge $\lambda$ on weights effect

In [ ]:
# plot weights with lambda regularization
weights_name = ['{:2d} th poly'.format(d) for d in np.arange(1, degree + 1)]
weights = weights[:, 1:] # remove bias term
plot_lambda_weights_effect(weights, lambda_vec, weights_name)

In [ ]:
# find lambda with lowest train and valid rmse, skip lambda = 0
opt_lambda = lambda_vec[np.argmin(np.abs(train_rmse[1:] - valid_rmse[1:])) + 1]
data2_train_X_tr = preprocess_pipe.transform(data2_train_X)
model.alpha = opt_lambda
model.fit(data2_train_X_tr, data2_train_y)

print("optimal lambda {:2.6e}".format(opt_lambda))
plot_model_result(model, preprocess_pipe, data2_train_set, data2_valid_set)
show_model_result(model, preprocess_pipe, data2_train_set, data2_valid_set, data2_test_set)

### Fit the data using 16th order polynomial regression model with scikit-learn Lasso model.

In [ ]:
# parameters
degree = 16

grid = np.linspace(-2, 5, 50)
lambda_vec = np.array([0] + [np.power(10.0, p) for p in grid])
#lambda_vec = np.r_[0, np.logspace(-2, 5, 20)].reshape(-1, 1)

preprocess_pipe = create_poly_feature_pipe(degree, normalize=True)
preprocess_pipe.fit(data2_train_X)

model = Lasso()
train_rmse, valid_rmse, weights = compute_validate_curve(model, preprocess_pipe, 
                                                data2_train_set, data2_valid_set, lambda_vec)
plot_validate_curve(train_rmse, valid_rmse, lambda_vec)

#### scikit-learn Lasso $\lambda$ on weights effect

In [ ]:
# plot weights with lambda regularization
weights_name = ['{:2d} th poly'.format(d) for d in np.arange(1, degree + 1)]
weights = weights[:, 1:] # remove bias term
plot_lambda_weights_effect(weights, lambda_vec, weights_name)

In [ ]:
# find lambda with lowest train and valid rmse, skip lambda = 0
opt_lambda = lambda_vec[np.argmin(np.abs(train_rmse[1:] - valid_rmse[1:])) + 1]
data2_train_X_tr = preprocess_pipe.transform(data2_train_X)
model.alpha = opt_lambda
model.fit(data2_train_X_tr, data2_train_y)

print("optimal lambda {:2.6e}".format(opt_lambda))
plot_model_result(model, preprocess_pipe, data2_train_set, data2_valid_set)
show_model_result(model, preprocess_pipe, data2_train_set, data2_valid_set, data2_test_set)

## 3. Regularization with Tensorflow


In [ ]:
df = pd.read_csv('ex2data3.csv')
df = df.drop('Unnamed: 0', axis=1)

split_train, split_valid, split_test = train_val_test_split(df, 0.6, 0.2, 0.2)

data3_train_X, data3_train_y = split_train.iloc[:, 0:-1].values.reshape(-1, 8), \
                               split_train.iloc[:, -1].values.reshape(-1, 1) 
data3_valid_X, data3_valid_y = split_valid.iloc[:, 0:-1].values.reshape(-1, 8), \
                               split_valid.iloc[:, -1].values.reshape(-1, 1)
data3_test_X ,  data3_test_y = split_test.iloc[:, 0:-1].values.reshape(-1, 8), \
                               split_test.iloc[:, -1].values.reshape(-1, 1)

data3_train_set = (data3_train_X, data3_train_y)
data3_valid_set = (data3_valid_X, data3_valid_y)
data3_test_set  = (data3_test_X, data3_test_y)


### Fit the training data using regression model with ridge (L2 penalty) regularization with scikit-learn Ridge model.

In [ ]:
# parameters
degree = 1

grid = np.linspace(-2, 5, 50)
lambda_vec = np.array([0] + [np.power(10.0, p) for p in grid])
#lambda_vec = np.r_[0, np.logspace(-2, 5, 20)].reshape(-1, 1)

preprocess_pipe = create_poly_feature_pipe(degree, normalize=True)
preprocess_pipe.fit(data3_train_X)

model = Ridge()
train_rmse, valid_rmse, weights = compute_validate_curve(model, preprocess_pipe, 
                                                data3_train_set, data3_valid_set, lambda_vec)
plot_validate_curve(train_rmse, valid_rmse, lambda_vec)

#### scikit-learn Ridge $\lambda$ on weights effect

In [ ]:
# plot weights with lambda regularization
weights_name = df.columns.values[0:-1]
weights = weights[:, 1:] # remove bias term
plot_lambda_weights_effect(weights, lambda_vec, weights_name)

In [ ]:
# find lambda with lowest train and valid rmse, skip lambda = 0
opt_lambda = lambda_vec[np.argmin(np.abs(train_rmse[1:] - valid_rmse[1:])) + 1]
data3_train_X_tr = preprocess_pipe.transform(data3_train_X)
model.alpha = opt_lambda
model.fit(data3_train_X_tr, data3_train_y)

print("optimal lambda {:2.6e}".format(opt_lambda))
show_model_result(model, preprocess_pipe, data3_train_set, data3_valid_set, data3_test_set)

### Fit the training data using regression model with lasso (L1 penalty) regularization with scikit-learn Lasso model.

In [ ]:
# parameters
degree = 1

grid = np.linspace(-2, 5, 50)
lambda_vec = np.array([0] + [np.power(10.0, p) for p in grid])
#lambda_vec = np.r_[0, np.logspace(-2, 5, 20)].reshape(-1, 1)

preprocess_pipe = create_poly_feature_pipe(degree, normalize=True)
preprocess_pipe.fit(data3_train_X)

model = Lasso()
train_rmse, valid_rmse, weights = compute_validate_curve(model, preprocess_pipe, 
                                                data3_train_set, data3_valid_set, lambda_vec)
plot_validate_curve(train_rmse, valid_rmse, lambda_vec)

Lasso train and valid set error is very small, its difficult to viuslaize the difference on graph

In [ ]:
plt.figure()
plt.title('Log scale RMSE error')
plt.semilogy(lambda_vec, train_rmse, color='g', label='Train')
plt.semilogy(lambda_vec, valid_rmse, color='b', label='Valid')
plt.xlabel('$lambda$')
plt.ylabel('RMSE')
plt.legend(loc='upper right')
plt.show()

#### scikit-learn Lasso $\lambda$ on weights effect

In [ ]:
# plot weights with lambda regularization
weights_name = df.columns.values[0:-1]
weights = weights[:, 1:] # remove bias term
plot_lambda_weights_effect(weights, lambda_vec, weights_name)

In [ ]:
# find lambda has lowest difference between train and valid rmse
opt_lambda = lambda_vec[np.argmin(np.abs(train_rmse[1:] - valid_rmse[1:])) + 1]
data3_train_X_tr = preprocess_pipe.transform(data3_train_X)
model.alpha = opt_lambda
model.fit(data3_train_X_tr, data3_train_y)

print("optimal lambda {:2.6e}".format(opt_lambda))
show_model_result(model, preprocess_pipe, data3_train_set, data3_valid_set, data3_test_set)

### Fit the training data using regression model with ridge (L2 penalty) regularization using TensorFlow.

In [ ]:
import tensorflow as tf

In [ ]:
class TFGradientDescentSolver(_BaseSolver, RegressorMixin):
    def __init__(self, alpha=1.0, add_bias=True,
                 learning_rate=1e-2, max_iter=None, tol=1e-8, regularizer='Ridge'):
        self.alpha = alpha
        self.add_bias = add_bias
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.tol = tol
        self.regularizer = regularizer
        
    def fit(self, X, y, sess):
        X, y = preprocess_data(X, y, self.add_bias)
        
        self.coef_, self.n_iter_, self.costs_ = \
            tf_gradient_descent(sess, X, y, 
                                lambda_=self.alpha, learning_rate=self.learning_rate,
                                tol=self.tol, max_iter=self.max_iter, regularizer=self.regularizer)
        

def tf_gradient_descent(sess, X, y, lambda_=1, learning_rate=1e-2, tol=1e-8, max_iter=None, regularizer='Ridge'):
    n_samples, n_features = X.shape
    
    if max_iter is None:
        max_iter = 1000
    
    mask = np.ones([n_features, 1])
    mask[0] = 0
    costs = np.zeros(max_iter)
    
    # TF variables
    X_ = tf.placeholder(tf.float32, [None, n_features])
    y_ = tf.placeholder(tf.float32, [None, 1])
    learning_rate_ = tf.constant(learning_rate)
    reg_lambda_ = tf.constant(lambda_, dtype=tf.float32)
    mask_ = tf.constant(mask, dtype=tf.float32)
    w_ = tf.Variable(tf.zeros([n_features, 1]))
    
    # cost function term
    diff_ = tf.matmul(X_, w_) - y
    obj_cost_ = tf.reduce_mean(tf.pow(diff_, 2)) / 2
    obj_grad_ = tf.matmul(tf.transpose(X_), diff_) / tf.cast(tf.shape(X_)[0], tf.float32)
    
    # regularization term
    if regularizer == "Ridge":
        reg_cost_ = tf.reduce_sum(tf.pow(mask_ * w_, 2)) / (2 * tf.cast(tf.shape(X_)[0], tf.float32))
        reg_grad_ = (mask_ * w_) / tf.cast(tf.shape(X_)[0], tf.float32)
        cost_ = obj_cost_ + reg_lambda_ * reg_cost_
        grad_ = obj_grad_ + reg_lambda_ * reg_grad_
        
    elif regularizer == "Lasso":
        # FIXME: Lasso soft_threshold implementation is wrong
        #        Using tf.gradient to replace compute l1 gradient
        reg_cost_ = tf.reduce_sum(tf.abs(mask_ * w_)) / (2 * tf.cast(tf.shape(X_)[0], tf.float32))
        #reg_grad_ = w_ / (2 * tf.cast(tf.shape(X_)[0], tf.float32))
        cost_ = obj_cost_ + reg_lambda_ * reg_cost_
        #grad_ = obj_grad_ + reg_lambda_ * reg_grad_
        grad_ = tf.gradients(cost_, xs=w_)
        
    # gradient update step
    train_op = tf.assign(w_, tf.reshape(w_ - learning_rate_ * grad_, [n_features, 1]))
    
    feed_dict = {X_: X, y_: y}
    
    sess.run(tf.assign(w_, tf.zeros([n_features, 1])))
    costs[0] = sess.run(cost_, feed_dict)
    for n_iter in np.arange(1, max_iter):
        _ = sess.run(train_op, feed_dict)
        costs[n_iter] = sess.run(cost_, feed_dict)
        
        #if n_iter % 50 == 0:
        #    costs[n_iter] = sess.run(cost_, feed_dict)
        #    print('costs[%4d]: %.8f' % (n_iter, costs[n_iter]))
        if np.abs(costs[n_iter - 1] - costs[n_iter]) < tol:
            costs = costs[:n_iter]
            break
    w = sess.run(w_)

    return w, n_iter, costs
    

In [ ]:
degree = 1

grid = np.linspace(-2, 5, 50)
lambda_vec = np.array([0] + [np.power(10.0, p) for p in grid])
#lambda_vec = np.r_[0, np.logspace(-2, 5, 20)].reshape(-1, 1)

train_rmse = np.zeros(len(lambda_vec))
valid_rmse = np.zeros(len(lambda_vec))

preprocess_pipe = create_poly_feature_pipe(degree, normalize=True)
data3_train_X_tr = preprocess_pipe.fit_transform(data3_train_X)
data3_valid_X_tr = preprocess_pipe.transform(data3_valid_X)

with tf.Session() as sess:
    
    init = tf.global_variables_initializer()
    sess.run(init)
        
    weights = []
    for idx, lambda_ in enumerate(lambda_vec):

        model = TFGradientDescentSolver(lambda_, learning_rate=0.1, max_iter=2000)
        model.fit(data3_train_X_tr, data3_train_y, sess)
        weights.append(extract_model_weights(model))
        
        train_rmse[idx] = compute_rmse(data3_train_y, model.predict(data3_train_X_tr))
        valid_rmse[idx] = compute_rmse(data3_valid_y, model.predict(data3_valid_X_tr))
    
weights = np.squeeze(np.array(weights))
    
plot_validate_curve(train_rmse, valid_rmse, lambda_vec)

#### TensorFlow Gradient Descent with ridge (L2 penalty) regularization $\lambda$ on weights effect

In [ ]:
# plot weights with lambda regularization
weights_name = df.columns.values[:-1]
weights = weights[:, 1:] # remove bias term
plot_lambda_weights_effect(weights, lambda_vec, weights_name)

In [ ]:
# find lambda has lowest difference between train and valid rmse
opt_lambda = lambda_vec[np.argmin(np.abs(train_rmse[1:] - valid_rmse[1:])) + 1]
data3_train_X_tr = preprocess_pipe.transform(data3_train_X)
model.alpha = opt_lambda

with tf.Session() as sess:
    model.fit(data3_train_X_tr, data3_train_y, sess)

print("optimal lambda {:2.6e}".format(opt_lambda))
show_model_result(model, preprocess_pipe, data3_train_set, data3_valid_set, data3_test_set)

## Fit the training data using regression model with lasso (L1 penalty) regularization using TensorFlow.

In [ ]:
degree = 1

grid = np.linspace(-2, 5, 50)
lambda_vec = np.array([0] + [np.power(10.0, p) for p in grid])
#lambda_vec = np.r_[0, np.logspace(-2, 5, 20)].reshape(-1, 1)

train_rmse = np.zeros(len(lambda_vec))
valid_rmse = np.zeros(len(lambda_vec))

preprocess_pipe = create_poly_feature_pipe(degree, normalize=True)
data3_train_X_tr = preprocess_pipe.fit_transform(data3_train_X)
data3_valid_X_tr = preprocess_pipe.transform(data3_valid_X)

with tf.Session() as sess:
    
    init = tf.global_variables_initializer()
    sess.run(init)

    weights = []
    for idx, lambda_ in enumerate(lambda_vec):

        model = TFGradientDescentSolver(lambda_, regularizer='Lasso', max_iter=3000)
        model.fit(data3_train_X_tr, data3_train_y, sess)
        weights.append(extract_model_weights(model))
    
        train_rmse[idx] = compute_rmse(data3_train_y, model.predict(data3_train_X_tr))
        valid_rmse[idx] = compute_rmse(data3_valid_y, model.predict(data3_valid_X_tr))

weights = np.squeeze(np.array(weights))    
    
plot_validate_curve(train_rmse, valid_rmse, lambda_vec)

#### TensorFlow Gradient Descent with lasso (L1 penalty) regularization $\lambda$ on weights effect

In [ ]:
# plot weights with lambda regularization
weights_name = df.columns.values[:-1]
weights = weights[:, 1:] # remove bias term
plot_lambda_weights_effect(weights, lambda_vec, weights_name)

In [ ]:
# find lambda has lowest difference between train and valid rmse
opt_lambda = lambda_vec[np.argmin(np.abs(train_rmse[1:] - valid_rmse[1:])) + 1]
data3_train_X_tr = preprocess_pipe.transform(data3_train_X)
model.alpha = opt_lambda

with tf.Session() as sess:
    model.fit(data3_train_X_tr, data3_train_y, sess)

print("optimal lambda {:2.6e}".format(opt_lambda))
show_model_result(model, preprocess_pipe, data3_train_set, data3_valid_set, data3_test_set)